In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

df_employees = pd.read_excel('/home/wissal/Téléchargements/Updated_Employees_Data.xlsx')  # Adjust the path
df_tasks = pd.read_excel('/home/wissal/Téléchargements/New_Tasks_Data.xlsx')  # Adjust the path

merged_df = pd.merge(df_employees, df_tasks, left_on='Skills', right_on='Required_Skill')

tasks_features = merged_df['Assembly_Line'].apply(lambda x: [f"Assembly_Line_{i}" for i in range(x)])
tasks_features = tasks_features.str.join(',').str.get_dummies(',')

employee_features = pd.concat([merged_df['Available_Days'].apply(lambda x: [f"Day_{day}" for day in x]).str.join(',').str.get_dummies(','),
                              merged_df['Available_Shifts'].apply(lambda x: [f"Shift_{shift}" for shift in x]).str.join(',').str.get_dummies(','),
                              merged_df['Skills'].str.get_dummies(',')],
                             axis=1)

X = pd.concat([employee_features, tasks_features], axis=1)

y = merged_df['Employee_ID']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = SVC()
clf.fit(X_train, y_train)

all_predictions = clf.predict(X)

merged_df['Assigned_Employee'] = all_predictions

selected_tasks = pd.DataFrame(columns=merged_df.columns)
selected_employees = set()

for _, task_row in merged_df.iterrows():
    if task_row['Employee_ID'] not in selected_employees:
        selected_employees.add(task_row['Employee_ID'])
        selected_tasks = pd.concat([selected_tasks, pd.DataFrame([task_row])], ignore_index=True)

        if len(selected_tasks) == 30:
            break

print(selected_tasks[['Task_ID', 'Required_Skill', 'Assembly_Line', 'Employee_ID', 'Assigned_Employee']])


  Task_ID     Required_Skill Assembly_Line Employee_ID Assigned_Employee
0   T_002          Packaging             1       E_019             E_019
1   T_002          Packaging             1       E_038             E_038
2   T_002          Packaging             1       E_051             E_051
3   T_007      Quality_Check             2       E_026             E_026
4   T_007      Quality_Check             2       E_027             E_027
5   T_007      Quality_Check             2       E_043             E_043
6   T_001  Machine_Operation             6       E_032             E_032
7   T_003           Assembly             6       E_041             E_041
